# The Final Trajectory

In order to compute the dynamics in SEOBNRv5HM, we will need to implement the corresponding equations of motion and a numerical integration routine.

The equations of motion require as inputs:
- The masses $m_{1,2}$ in normalized masses, i.e $m_1 + m_2 = 1$.
- The spins $\chi_1$, and $\chi_2$ of the binary in dimensionless units.
- The values of the primitive variables $(r,\phi,p_{r_*},p_{\phi})$.
It then computes:
- The partial derivatives of the Hamiltonian w.r.t the positions, $\tfrac{\partial H}{\partial r}$ and momenta $\tfrac{\partial H}{\partial p_{(r_*,\phi)}}$.
- The tortoise parameter $\xi$.
- The Hamiltonian $H$, the circular frequency $\Omega_{\rm circ}$ that are then used to solve for the flux $F_{\phi}$
And returns the time derivative of each of the primitive variables given by Hamilton's equations of motion as well as the prescription for the radiation-reaction.

The numerical integration routine is split into two parts
- An initial sparsely sampled integration that stops at given stopping conditions
- A finely-sampled interpolation of the initial integration from some stepback time to the stopping conditions to calculate the Non-Quasi-Circular (NQC) Coefficients

In this notebook, we will generate three files:
- The set of expressions needed to generate the equations of motions
- The numerical routines needed to generate the trajectory

Finally, the numerical integration requires as inputs:
- The initial conditions, documented in the initial conditions routine
- The stopping conditions, documented in this notebook
- All other inputs needed for the equations of motion 

In [1]:
import sys,os#Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

# Create C code output directory:
Ccodesdir = "Dynamics"
# Then create an output directory in case it does not exist
cmd.mkdir(Ccodesdir)

# Step : The Equations of Motions

The equations of motion are given by:

\begin{equation*}
\dot{r} = \frac{1}{\nu}\xi \frac{\partial H}{\partial p_{r_{*}}}\\
\dot{\phi} = \Omega\\
\dot{p}_{r_*} = \frac{1}{\nu}\left(-\xi \frac{\partial H}{\partial r} + \frac{ p_{ r_{*} }F_{\phi} }{ p_{\phi} }\right)\\
\dot{p}_{\phi} = \frac{1}{\nu}F_{\phi}
\end{equation*}

Where, $\nu$ is the symmetric mass ratio, $F_{\phi}$ is the gravitational wave flux, $\frac{\partial H}{\partial \zeta}$ is the partial derivative of the Hamiltonian, $H$, with respect to the primitive variable $\zeta = \{r,\phi,p_{r_{*}},p_\phi\}$.  

In [2]:
%%writefile $Ccodesdir/v5HM_Equations_Of_Motion.txt
rdot = (xi/eta)*dHdprstar(m1,m2,r,prstar,pphi,chi1,chi2)
phidot = Omega
prstardot = (1/eta)*(-xi*dHdr(m1,m2,r,prstar,pphi,chi1,chi2) + prstar*F_phi/pphi)
pphidot = F_phi/eta

Overwriting Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $F_{\phi}$

The gravitational wave flux is given as a function of the primitive variables $\{r,\phi,p_{r_{*}},p_{\phi}\}$, the masses $\{m_1,m_2\}$, the spins $\{\chi_{1},\chi_{2}\}$, the rotational frequency $\Omega$, the circular frequency $\Omega_{\rm circ}$, and the Hamiltonian $H$.

$$
F_{\phi} \equiv F_{\phi}(m_1,m_2,r,p_{r_{*}},p_{\phi},\chi_1,\chi_2,\Omega,\Omega_{\rm circ},H)
$$

In [3]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
F_phi = flux(m1, m2, r, phi, prstar, pphi, chi1, chi2,Omega,Omega_circ,H)/Omega

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\Omega$

The rotational frequency is given by:

$$
\Omega = \frac{1}{\nu}\frac{\partial H}{\partial p_{\phi}}
$$

In [4]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
Omega = (1/eta)*dHdpphi(m1,m2,r,prstar,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\Omega_{\rm circ}$

The circular frequency is given by:

$$
\Omega = \frac{1}{\nu}\left.\frac{\partial H}{\partial p_{\phi}}\right|_{p_{r_{*}} = 0}
$$

In [5]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
Omega_circ = (1/eta)*dHdpphi_preq0(m1,m2,r,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $H$, $\xi$

The Hamiltonian and the tortoise parameter are given as the result of the Hamiltonian computation, which in turn is a function of the primitive variables and the input parameters

In [6]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
H , xi = Hamiltonian(m1,m2,r,prstar,pphi,chi1,chi2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : $\eta$

The symmetric mass ratio is defined as follows

$$
\eta = \frac{m_{1}m_{2}}{(m_1 + m_2)^2}
$$

In [7]:
%%writefile -a $Ccodesdir/v5HM_Equations_Of_Motion.txt
eta = m1*m2/(m1 + m2)/(m1 + m2)

Appending to Dynamics/v5HM_Equations_Of_Motion.txt


# Step : Write into python function

In [8]:
with open(os.path.join(Ccodesdir,"v5HM_Equations_Of_Motion.py"), "w") as output:
    output.write("import numpy as np\n")
    output.write("from Radiation.v5HM_Flux_unoptimized import v5HM_unoptimized_flux as flux\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_omega_circ as dHdpphi_preq0\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dpphi as dHdpphi\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dprstar as dHdprstar\n")
    output.write("from Derivatives.v5HM_Hamiltonian_Derivatives_unoptimized import v5HM_unoptimized_dH_dr as dHdr\n")
    output.write("from Hamiltonian.v5HM_Hamiltonian_unoptimized import v5HM_unoptimized_hamiltonian as Hamiltonian\n")
    output.write("def v5HM_unoptimized_rhs(t,y,m1,m2,chi1,chi2,verbose = False):\n    r , phi , prstar , pphi = y[0] , y[1] , y[2] , y[3]\n")
    for line in reversed(list(open(os.path.join(Ccodesdir,"v5HM_Equations_Of_Motion.txt"),"r"))):
        output.write("    %s\n" % line.rstrip())
    output.write("    if not verbose:\n        return np.array([rdot,phidot,prstardot,pphidot])\n")
    output.write("    else:\n        return np.array([rdot,phidot,prstardot,pphidot]), F_phi, Omega, Omega_circ, xi, dHdr(m1,m2,r,prstar,pphi,chi1,chi2)/eta, eta, prstar,pphi\n")

import numpy as np
from Dynamics.v5HM_Equations_Of_Motion import v5HM_unoptimized_rhs as rhs_bob
from Dynamics.pyseobnr_equations_of_motion import get_rhs as rhs_true

N = 100000
gt_pert_total = [[],[],[],[]]
gt_pert_O1 = [[],[],[],[]]
gt_pert_O2 = [[],[],[],[]]
gt_pert_gtO3 = [[],[],[],[]]
rng = np.random.default_rng(seed = 50)
eta = rng.random(N)*.25
chi1 = 2.*rng.random(N)-1
chi2 = 2.*rng.random(N)-1
m2 = (1 - np.sqrt(1 - 4*eta))*.5
m1 = (1 + np.sqrt(1 - 4*eta))*.5
r = rng.random(N)*17. + 3.
phi = rng.random(N)*2.*np.pi
prstar = rng.random(N)*20. - 10.
pphi = rng.random(N)*20. - 10.

pert_exponent = 1e-14
pert_sign = 2*(rng.integers(0,high = 1,size = N)) - 1
pert_mantissa = 3.*rng.random(N) + 1.
pert = 1. + pert_sign*pert_mantissa*pert_exponent
chi1pert = chi1*pert 
chi2pert = chi2*pert
m2pert = m2*pert
m1pert = m1*pert
rpert = r*pert
phipert = phi*pert
prstarpert = prstar*pert
pphipert = pphi*pert


disagrmt_max = [0,0,0,0]
disagrmt_max_index = [0,0,0,0]
nans = []
def E_rel(a,b):
    return np.abs((a - b)/a)

for i in range(N):
    dyn_bob = rhs_bob(0.,[r[i],phi[i],prstar[i],pphi[i]],m1[i],m2[i],chi1[i],chi2[i])
    dyn_true = rhs_true(0.,[r[i],phi[i],prstar[i],pphi[i]],chi1[i],chi2[i],m1[i],m2[i])
    dyn_pert = rhs_true(0.,[rpert[i],phipert[i],prstarpert[i],pphipert[i]],chi1pert[i],chi2pert[i],m1pert[i],m2pert[i])
    e_rels = [E_rel(dyn_bob[0],dyn_true[0]),E_rel(dyn_bob[1],dyn_true[1]),E_rel(dyn_bob[2],dyn_true[2]),E_rel(dyn_bob[3],dyn_true[3])]
    tols = [E_rel(dyn_pert[0],dyn_true[0]),E_rel(dyn_pert[1],dyn_true[1]),E_rel(dyn_pert[2],dyn_true[2]),E_rel(dyn_pert[3],dyn_true[3])]
    for j in range(4):
        e_rel , tol = e_rels[j] , tols[j]
        if (e_rel>tol and tol > 0):
            gt_pert_total[j].append(i)
            if e_rel/tol > disagrmt_max[j]:
                disagrmt_max[j] = e_rel/tol
                disagrmt_max_index[j] = i
            if e_rel > 1000*tol:
                gt_pert_gtO3[j].append(i)
            elif e_rel > 100*tol:
                gt_pert_O2[j].append(i)
            elif e_rel > 10*tol:
                gt_pert_O1[j].append(i)
dvar = ['r','phi','prstar','pphi']
for j in range(4):
    print("Of total ",str(N)," comparisons, for %s\n"%dvar[j],
          "number of cases with relative error (total)  greater than allowed: ",len(gt_pert_total[j]),"\n",
          "number of cases with relative error O(10)    greater than allowed: ",len(gt_pert_O1[j]),"\n",
          "number of cases with relative error O(100)   greater than allowed: ",len(gt_pert_O2[j]),"\n",
          "number of cases with relative error O(1000+) greater than allowed: ",len(gt_pert_gtO3[j]))
#results = np.array([eta,r,phi,prstar,pphi,chi1,chi2])
#analytics = np.array([[len(gt_pert_total[0]),len(gt_pert_total[1]),len(gt_pert_total[2]),len(gt_pert_total[2])],[len(gt_pert_O1[0]),len(gt_pert_O1[1]),len(gt_pert_O1[2])],[len(gt_pert_O2[0]),len(gt_pert_O2[1]),len(gt_pert_O2[2])],[len(gt_pert_gtO3[0]),len(gt_pert_gtO3[1]),len(gt_pert_gtO3[2])],disagrmt_max,disagrmt_max_index])
#np.savetxt(os.path.join(outputdir,"eom_validation_results.dat"),results)
#np.savetxt(os.path.join(outputdir,"eom_validation_analytics.dat"),results)
exit(1)

C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:16: RuntimeWarning: invalid value encountered in scalar power
  vomega = Omega**(np.divide(1,3))
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:17: RuntimeWarning: invalid value encountered in scalar power
  vphi = Omega*(Omega_circ**(-np.divide(2,3)))
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:348: RuntimeWarning: invalid value encountered in log
  T55 = gamma(5 + 1 - 2*1j*khat5)*np.exp(np.pi*khat5)*(np.exp(2*1j*khat5*np.log(2*5*Omega*r0)))/factorial(5)
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:352: RuntimeWarning: invalid value encountered in log
  T43 = gamma(4 + 1 - 2*1j*khat3)*np.exp(np.pi*khat3)*(np.exp(2*1j*khat3*np.log(2*3*Omega*r0)))/factorial(4)
C:\Users\sidmahesh\Documents\GitHub\bob\V5\Radiation\v5HM_Flux_unoptimized.py:354: RuntimeWarning: invalid value encountered in log
  T44 = gamma(4 + 1 - 2*

Of total  100000  comparisons, for r
 number of cases with relative error (total)  greater than allowed:  1 
 number of cases with relative error O(10)    greater than allowed:  0 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
Of total  100000  comparisons, for phi
 number of cases with relative error (total)  greater than allowed:  418 
 number of cases with relative error O(10)    greater than allowed:  7 
 number of cases with relative error O(100)   greater than allowed:  0 
 number of cases with relative error O(1000+) greater than allowed:  0
Of total  100000  comparisons, for prstar
 number of cases with relative error (total)  greater than allowed:  10190 
 number of cases with relative error O(10)    greater than allowed:  1564 
 number of cases with relative error O(100)   greater than allowed:  127 
 number of cases with relative error O(1000+) greater than allowed:  13
Of total 

# Step : The Stopping Conditions

The stopping conditions are given in lines 152-178 of [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py) of pyseobnr.

There are several termination conditions that are checked when $r < 6$:

- Peak orbital frequency : If the value of $\Omega$ has decreased since the last timestep.
- Positive radial velocity: If $\dot{r} > 0$
- Positive radial momentum derivative: If $\dot{p_{r_{*}}} > 0$
- Radius reaches threshold: If the radius reaches some threshold radius (conditionally defined by numerical relativity and the Kerr metric last stable circular orbit)
- Unphysical circular frequency at low $r$'s: if $r < 3M$ and $\Omega_{\rm circ} > 1$

We will define each of these conditions below.

# Step : The First Integration

In the first integration, the initial conditions are evaluated and the trajectory is evolved until the above specified stopping conditions are accomplished. 
pyseobnr uses `pygsl_lite` to access gsl's `odeiv2` tools. We use the equivalent `scipy.odeint` tools.

We use an adaptive Runge-Kutta 45 method with relative tolerance `1e-11` and absolute tolerance `1e-12`

In [9]:
validation = False
if validation:
    import time
    from Dynamics.v5HM_Initial_Conditions import v5HM_initial_conditions as IC
    from scipy.integrate import solve_ivp
    event = lambda t,y,m1,m2,chi1,chi2 : y[0] - 6.0
    event.terminal = True

    M = 33.
    q = 23./10.
    S1 = 0.5
    S2 = 0.9
    f = 20
    m1 = q/(1 + q)
    m2 = 1/(1+q)
    event = lambda t,y,a,b,c,d : y[0] - 6.
    event.terminal = True
    yinit = IC(M,q,S1,S2,f)
    sol_bob = solve_ivp(rhs_bob,[0.,20000.],yinit,method = 'RK45',events = [event], args = (m1,m2,S1,S2), rtol = 1e-11, atol = 1e-12)
    sol_true = solve_ivp(rhs_true,[0.,20000.],yinit,method = 'RK45',events = [event], args = (S1,S2,m1,m2), rtol = 1e-11, atol = 1e-12)

In [10]:
if validation:
    import matplotlib.pyplot as plt
    import numpy as np
    t_bob = sol_bob.t
    t_true = sol_true.t
    r_bob = sol_bob.y[0]
    r_true = sol_true.y[0]
    plt.scatter(t_bob,r_bob,s = 1,label = 'ours',color = 'black')
    plt.plot(t_true,r_true,label = 'pyseobnr')
    plt.xlabel(r'time($M$)')
    plt.ylabel(r'$r$($M$)')
    plt.legend()
    plt.savefig('first_trajectory_eta_1_chi1_5_chi2_9.png',dpi = 300)

# Step : Implement the pyseobnr integration routine

In these sections we will document pyseobnr's exact method for integrating the trajectory.

# Step : Declare the gsl ODE solver specifications

In this section, we document the steps needed to set up the ODE solver in gsl. For more details on gsl's `odeiv2` package see [this tutorial](https://www.gnu.org/software/gsl/doc/html/ode-initval.html)
pyseobnr uses the following specifications:

- step type : rk8pd (Dormand-Prince adaptive 8th order Runge-Kutta method)
- absolute tolerance : `1e-11`
- relative tolerance : `1e-12`

In [11]:
%%writefile $Ccodesdir/v5HM_Integrator.txt

sys = odeiv2.system(v5HM_unoptimized_RHS,None,4,[m1,m2,chi1,chi2])
T = odeiv2.step_rk8pd
s = odeiv2.pygsl_lite_odeiv2_step(T,4)
atol = 1e-11
rtol = 1e-12
c = odeiv2.pygsl_lite_odeiv2_control(atol,rtol,a_y = 1,a_dydt = 1,None)
e = odeiv2.pygsl_lite_odeiv2_evolve(4)

Overwriting Dynamics/v5HM_Integrator.txt


# Step : Compute initial conditions and step-size

In this step we will call the initial conditions routine and use the stepsize choice given in line 117 of pyseobnr's [integrate_ode.py](https://git.ligo.org/waveforms/software/pyseobnr/-/blob/main/pyseobnr/eob/dynamics/integrate_ode.py):

$$
h = \frac{2\pi}{5\Omega_0 }
$$

Where, $\Omega_0$ is the starting frequency in geometric units.

$\Omega_0$ is given by:

$$
\Omega_0 = MM_\odot\pi f
$$

Where, $M$ is the total mass in solar masses and $f$ is the starting gravitational wave frequency.

In [12]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

y_init = v5HM_unoptimized_initial_conditions(M,q,chi1,chi2,f)
Omega_0 = M*4.925491025543575903411922162094833998e-6*sp.pi*f
h = 2*sp.pi/5/Omega_0
nu = q/(1 + q)/(1 + q)
m1 = q/(1 + q)
m2 = 1/(1 + q)

Appending to Dynamics/v5HM_Integrator.txt


# Step : Declare lists to store results as well as stopping condition checks

In this step we declare lists to store the primitive variables at each point as well as variables for checking the stopping conditions.

Namely, we check for 

- whether the frequency $\Omega$ has peaked; the $\Omega$ from the previous timestep is stored for this purpose
- whether the radial momentum $p_{r_{*}}$ has peaked

In [13]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

prims = []
times = []
omega_previous = Omega_0
omega_peak = False
prstar_peak = False
times.append(0.)
prims.append(y_init)

Appending to Dynamics/v5HM_Integrator.txt


# Step : Begin the integration loop and take a step

In this step, we begin the integration loop and perform the first step, appending the results.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

while t < 2.0e9:
    status, t, h, y = e.apply(c,s,sys,t,t1,h,y)
    if status != errno.GSL_SUCCESS:
            print("break status", status)
            break
    prims.append(y)
    times.append(t)
    

# Step : Do a stopping condition check

In this step, we check for the stopping conditions. First, we check is the radius is less than 6M.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

    if r < = 6:

# Step : Compute the right hand sides

In this step, we compute the right hand sides in order to do the stopping condition checks.

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        rhs = v5HM_unoptimized_RHS(t,y,m1,m2,chi1,chi2)
        drdt = rhs[0]
        dphidt = rhs[1]
        dprstardt = rhs[2]

# Step : Check for a frequency peak

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if dphidt < omega_previous:
            omega_peak = True
            break

# Step : Check for a positive radial derivative

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if drdt > 0:
            break

# Step : Check for a positive radial momentum derivative

In [ ]:
%%writefile -a $Ccodesdir/v5HM_Integrator.txt

        if dprdt > 0:
            prstar_peak = True
            break

# Step : 